<img src='../images/bildungscampus_logo.png' width="40%" align="left" />
<img src='../images/hhn.png' width="25%" align="right" />

# Schritt 4: Action Execution
Masterarbeit - Sebastian Kahlert | Fakultät Wirtschaft und Verkehr | Wirtschaftsinformatik - Informationsmanagement und Data Science | WS 2021/22

<img src='../images/bar.png'/>

## Generierung von SQL-Queries zur Aktualisierung bekannter Informationen

Hier wird eine neue Funktion implementiert, die den ermittelten Intent und die extrahierten Informationen aus dem Slot Filling als Input nimmt und bereits bekannte Informationen in der Datenbank aktualisiert. So kann zum Beispiel die veraltete Telefinnummer oder E,ial Adresse der Professoren aktualisiert werden.

### 4.1. Import benötigter Bibliotheken

In [2]:
# import of necessary libraries
import nltk 
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import datefinder
import numpy
import tflearn 
import tensorflow
import random 
import json
import pickle
import spacy
import sqlite3
import re
import os
import sqlite3
from difflib import SequenceMatcher

### 4.2. Intent Classification

Das Neuronale Netzwerk wurde zuvor in einem separaten Notebook erstellt, trainert und gespeichert. Der folgende Code versucht das gespeciherte Neuronale Netzwerk zu laden. Falls dies nicht möglich sein sollte wird ein neues Neuronales Netzwerk erstellt und trainiert. Hierzu werden die Trainingsdaten "data.pickle" geladen und die Layer des Netztes definiert.

In [3]:
# open training data for neural net
with open("../2. Intent Classififcation/data.pickle", "rb") as f:
    words, labels, training, output = pickle.load(f)

In [4]:
# open saved model or implement new one if model does not exist yet
tensorflow.compat.v1.reset_default_graph()
    
# Creating the Neural Network
net = tflearn.input_data(shape= [None, len(training[0])]) #input layer Neurons = numer of words in training
net = tflearn.fully_connected(net, 8) #hidden layer fully connected with 8 neuron
net = tflearn.fully_connected(net, len(output[0]), activation="softmax" ) #output layer 6 Neurons = labels
net = tflearn.regression(net)

model = tflearn.DNN(net)

try:
    model.load(r"..\2. Intent Classififcation\model.tflearn")
except:
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save(r"..\2. Intent Classififcation\model.tflearn")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Restoring parameters from C:\Users\Sebi\OneDrive\Studium\Thesis_Chatbot\2. Intent Classififcation\model.tflearn


In [6]:
model

In [7]:
# transform input in numbers to make it readable for neural net
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
    
    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
    
    return numpy.array(bag)

In [9]:
        results = model.predict([bag_of_words("Herr Stern changed his number its +23 3433 733 3733", words)])[0] #output is just a probability for each label
        results_index = numpy.argmax(results) #index of greatest value
        intent = labels[results_index] #output is the most probable label
        print(intent)

extract_new_telephone


### 4.3. Slot Filling

Für die Extraktion der verschiedenen Zusatzinformationen müssen unteranderem bereits bekannte Namen aus der Datenbank extrahiert werden und in Listen gespeichert werden. Eine Funktion gleicht den Input des Benutzers mit den Namen ab, um zu prüfen ob der vom Benutzer genannte Professor bekannt ist oder nicht. Zusätzlich werden zuvor trainerte Custom NER Modelle geladen und die Slot Filling definiert.

#### 4.3.1. Vornamen

In [10]:
# get first names from database
conn = sqlite3.connect("../PROF_INFO_DB.db")
cur = conn.cursor()
cur.execute("select first_name FROM PROF_INFO_TABLE")
conn.commit()
first_names = cur.fetchall()
conn.close()

#first_names

In [11]:
# store first names in list
first_names_list = []
for i in range(len(first_names)):
    answer = " ".join(first_names[i])
    answer.strip()
    first_names_list.append(answer)

#first_names_list

#### 4.3.1. Nachnamen

In [12]:
# get last names from database
conn = sqlite3.connect("../PROF_INFO_DB.db")
cur = conn.cursor()
cur.execute("select last_name FROM PROF_INFO_TABLE")
conn.commit()
last_names = cur.fetchall()
conn.close()

#last_names

In [13]:
# store last names in list
last_names_list = []
for i in range(len(last_names)):
    answer = " ".join(last_names[i])
    answer.strip()
    last_names_list.append(answer)

#last_names_list

#### 4.3.3. Forschungsschwerpunkte und Studiengänge

In [14]:
#load the custom NER models
nlp_research_area = spacy.load(r"../3. Slot Filling/custom_ner_model_research_area")
nlp_study = spacy.load(r"../3. Slot Filling/custom_ner_model_study")

#### 4.3.4. Slot Filling Funktion

In [94]:
def slot_filling_update(inp, intent):
    # liste with intents that need information
    #intent_prof_contact=["prof_telephone_query_name", "prof_email_query_name", "prof_office_query_name", "prof_research_area_query_name", "prof_study_query_name"]
   # intent_generic_conversation=["greeting","greeting_response","courtesy_greeting","courtesy_greeting_response","real_name_query", "goodbye","task_response"]
    
    #get name of professor
    for last_name in last_names_list:
        for word in inp.split():
            if SequenceMatcher(None, last_name, word).ratio() >= 0.7:
                prof_name = last_name

    
    #new phone number
    if intent == "extract_new_telephone":
        #get the new number
        re_number_1 = r"\D[\d]{2}? [\d]{4} [\d]{3} [\d]{4}"
        re_number_2 = r"\D[\d]{2}? [\d]{4} [\d]{3} [\d]{3}"
        re_number_3 = r"\D[\d]{2} [\(][\d]{1}[\)] [\d]{4} [\d]{3} [\d]{4}"
        extracted_info = re.compile("(%s|%s|%s)" % (re_number_1, re_number_2, re_number_3)).findall(inp)      
        
    elif intent == "extract_new_email":
        extracted_info = re.findall('\S+@\S+', inp)
        
    elif intent == "extract_new_office":
        re_office_1 = r"[A-Z, a-z].\d{1}.\d{2}"
        re_office_2 = r"[A-Z, a-z].\d{3}"
        extracted_info = re.compile("(%s|%s)" % (re_office_1, re_office_2)).findall(inp)
        
    elif intent == "extract_new_research_area":
        doc = nlp_research_area(inp)
        for ent in doc.ents:
            if ent.label_ == "RESEARCH_AREA":
                extracted_info = ent.text  
            else:
                extracted_info = "Not found"
    
    elif intent == "extract_new_study":
        doc = nlp_study(inp)
        for ent in doc.ents:
            if ent.label_ == "STUDY":
                extracted_info = ent.text  
            else:
                extracted_info = "Not found"
    
    return prof_name, extracted_info

In [95]:
prof_name, new_info = slot_filling_update("Herr Stern has a new email. It is test@test.de ", "extract_new_email")

In [96]:
conn = sqlite3.connect("../PROF_INFO_DB.db")
cur = conn.cursor()


In [97]:
new_info = "BWL"
prof_name = "Stern"

In [98]:
cur.execute("INSERT INTO PROF_RESEARCH_AREA_TABLE (prof_id,Name,research_area) VALUES((SELECT prof_id FROM PROF_INFO_TABLE WHERE last_name = ? ),?,?)", (prof_name, new_info, prof_name,))
conn.commit()

In [99]:
"SELECT prof_id FROM PROF_INFO_TABLE INNER JOIN PROF_RESEARCH_AREA_TABLE on PROF_RESEARCH_AREA_TABLE.prof_id = PROF_INFO_TABLE.prof_id WHERE Name = ? COLLATE NOCASE", (condition,))

SyntaxError: invalid syntax (<ipython-input-99-6836f440c7b0>, line 1)

###  4.4. Action Execution 

Je nach intent wird ein vordefiniertes SQL-Query mit Platzhalter ausgeführt. Zuvor müssen die Platzhalter mit den extrahierten Informationen aus dem Slot Filling befüllt werden. Output ist die Information aus der Datenbank

In [100]:
def action_execution_update(intent, prof_name, new_info):

    print(prof_name)
    print(new_info)
    #connect to database
    conn = sqlite3.connect("../PROF_INFO_DB.db")
    cur = conn.cursor()
    
    #TELEPHONE
    if intent == "extract_new_telephone":
        cur.execute("UPDATE PROF_INFO_TABLE SET telephone = ? WHERE last_name = ? COLLATE NOCASE", (new_info[0], prof_name,))
        conn.commit()

    elif intent == "extract_new_email": 
        cur.execute("UPDATE PROF_INFO_TABLE SET email = ? WHERE last_name = ? COLLATE NOCASE", (new_info[0], prof_name,))
        conn.commit()
        
    elif intent == "extract_new_office":
        cur.execute("UPDATE PROF_INFO_TABLE SET office = ? WHERE last_name = ? COLLATE NOCASE", (new_info[0], prof_name,))
        conn.commit()
        
    elif intent == "extract_new_research_area":
        #print("pog", condition[0])
        cur.execute("INSERT INTO PROF_RESEARCH_AREA_TABLE (prof_id,Name,research_area) VALUES((SELECT prof_id FROM PROF_INFO_TABLE WHERE last_name = ? ),?,?)", (prof_name, prof_name, new_info,))
        conn.commit()
    
    elif intent == "extract_new_study":
        #print("pog", condition[0])
        cur.execute("INSERT INTO PROF_STUDY_TABLE (prof_id,Name,study) VALUES((SELECT prof_id FROM PROF_INFO_TABLE WHERE last_name = ? ),?,?)", (prof_name, prof_name, new_info,))
        conn.commit()
    
    return answer

### 4.5. Testen der Action Execution Funktion

In [101]:
def chat():
    log = []
    print("Start talking with me!(type quit to stop):")
    while True:
        inp = input("You: ")
        log.append(inp) #saves all input of user in list
        if inp.lower() == "quit":
            print("Goodbye :)")
            break
        
        # predict the intent
        results = model.predict([bag_of_words(inp, words)])[0] #output is just a probability for each label
        results_index = numpy.argmax(results) #index of greatest value
        intent = labels[results_index] #output is the most probable label
        print(intent)
        prof_name, new_info = slot_filling(inp, intent)
                
        action_execution_update(intent, prof_name, new_info)
        
        
        #print("condition-->", extracted_info)
        #rint("answer from db-->", answer)
        


    return log # returns list of the inputs

In [102]:
chat()

Start talking with me!(type quit to stop):
You: stern has a new study it is software engineering
extract_new_study
Stern
software engineering


KeyboardInterrupt: Interrupted by user